In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 주문봇이야. 
            
            
            """
            "이전 대화와 현재 고객이 입력한 메시지 모두를 꼼꼼히 파악하여 답변해줘."
            """
            튜플의 첫 번째 값이 모델 필드에 입력된 값을 의미해
            STATUS_CHOICES = (
                ('order', '주문 완료'),
                ('payment_completed', '입금 완료'),
                ('order_changed', '주문 변경'),
                ('order_canceled', '주문 취소'),
            )"""
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "사용자 ID: {user_id}\n사용자 입력 메시지{input}"),
    ]
)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [ ]:
model = ChatOpenAI()

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
from langchain_core.tools import tool
from products.models import Order, OrderStatus

In [ ]:
@tool
def get_recent_orders_by_status(user_id, status):
    """
    Retrieve the three most recent orders with a specific status for the given user.
    """

    user = User.objects.get(id=user_id)

    recent_orders = Order.objects.filter(
        user=user,
        order_status=status,
    ).prefetch_related(
        Prefetch('status_changes', queryset=OrderStatus.objects.filter(status=status))
    ).order_by('-created_at')[:3]

    print("recent_orders: ", recent_orders)
    return [order.to_dict() if hasattr(order, 'to_dict') else {
        "id": order.id,
        "user_id": order.user.id,
        "created_at": order.created_at.strftime('%Y-%m-%d %H:%M:%S'),
        "order_status": order.order_status,
        "order_items": [{
            "product_name": item.product.product_name,
            "quantity": item.quantity,
            "price": float(item.price)
        } for item in order.order_items.all()]
    } for order in recent_orders]
    
tools = [get_recent_orders_by_status]

In [ ]:
def determine_tool_usage(msg: AIMessage) -> Runnable:
    """Determine whether to use the tool based on the given condition."""
    print(msg)
    if msg.additional_kwargs: 
        tool_map = {tool.name: tool for tool in tools}
        tool_calls = msg.tool_calls.copy()
        for tool_call in tool_calls:
            tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
        print("도구 출력 결과: ", tool_call["output"])
        return tool_call["output"]
    else:
        return msg.content

In [ ]:
chain_with_tools = prompt | model.bind_tools(tools) | determine_tool_usage

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:

chain_with_tools_n_history  = RunnableWithMessageHistory(
    chain_with_tools,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

---

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | model

In [4]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [5]:
with_message_history.invoke(
    {"ability": "math", "input": "What does cosine mean?"},
    config={"configurable": {"session_id": "abc123"}},
)

Parent run cae0e862-d92c-46a4-8d4d-fa4805d94469 not found for run 5f885d83-b609-444f-9c92-c606f1fe4084. Treating as a root run.


AIMessage(content='Cosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle.', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 33, 'total_tokens': 59}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fde01fc1-f2d8-4409-a3a6-1c8a26a516ec-0')

In [8]:
store["abc123"].messages

[HumanMessage(content='What does cosine mean?'),
 AIMessage(content='Cosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle.', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 33, 'total_tokens': 59}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fde01fc1-f2d8-4409-a3a6-1c8a26a516ec-0')]

In [9]:
store["abc123"].pop()

AIMessage(content='Cosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle.', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 33, 'total_tokens': 59}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fde01fc1-f2d8-4409-a3a6-1c8a26a516ec-0')

In [13]:
store["abc123"].messages.append([{'id': 5, 'user_id': 1, 'created_at': '2024-05-13 02:15:33', 'order_status': 'order', 'order_items': [{'product_name': '떡케익5호', 'quantity': 1, 'price': 54000.0}]}])

In [14]:
store["abc123"].messages

[HumanMessage(content='What does cosine mean?'),
 [{'id': 5,
   'user_id': 1,
   'created_at': '2024-05-13 02:15:33',
   'order_status': 'order',
   'order_items': [{'product_name': '떡케익5호',
     'quantity': 1,
     'price': 54000.0}]}]]

In [16]:
with_message_history.invoke(
    {"ability": "math", "input": "What does sine mean?"},
    config={"configurable": {"session_id": "abc123"}},
)

Parent run d29ab4fc-da62-4b30-a4aa-928af0c0abd3 not found for run ba40ab90-59e0-49d0-bfc5-56e8c73c8627. Treating as a root run.


NotImplementedError: Unsupported message type: <class 'list'>

---